# Import library

In [ ]:
"""

Potential Field based path planner

author: Atsushi Sakai (@Atsushi_twi)

Ref:
https://www.cs.cmu.edu/~motionplanning/lecture/Chap4-Potential-Field_howie.pdf

"""

from collections import deque
import numpy as np
import matplotlib.pyplot as plt
import os

# Constant setting

In [ ]:
# Parameters
KP = 5.0  # attractive potential gain
ETA = 100.0  # repulsive potential gain
AREA_WIDTH = 30.0  # potential area width [m]
# the number of previous positions used to check oscillations
OSCILLATIONS_DETECTION_LENGTH = 3

show_animation = True

# Potential method

In [ ]:
def calc_potential_field(gx, gy, ox, oy, reso, rr, sx, sy):
    minx = min(min(ox), sx, gx) - AREA_WIDTH / 2.0
    miny = min(min(oy), sy, gy) - AREA_WIDTH / 2.0
    maxx = max(max(ox), sx, gx) + AREA_WIDTH / 2.0
    maxy = max(max(oy), sy, gy) + AREA_WIDTH / 2.0
    xw = int(round((maxx - minx) / reso))
    yw = int(round((maxy - miny) / reso))

    # calc each potential
    pmap = [[0.0 for i in range(yw)] for i in range(xw)]

    for ix in range(xw):
        x = ix * reso + minx

        for iy in range(yw):
            y = iy * reso + miny
            ug = calc_attractive_potential(x, y, gx, gy)
            uo = calc_repulsive_potential(x, y, ox, oy, rr)
            uf = ug + uo
            pmap[ix][iy] = uf

    return pmap, minx, miny


def calc_attractive_potential(x, y, gx, gy):
    return 0.5 * KP * np.hypot(x - gx, y - gy)  # np.hypot() ~ square(x**2+y**2)


def calc_repulsive_potential(x, y, ox, oy, rr):
    # search nearest obstacle
    minid = -1
    dmin = float("inf")
    for i, _ in enumerate(ox):
        d = np.hypot(x - ox[i], y - oy[i])
        if dmin >= d:
            dmin = d
            minid = i

    # calc repulsive potential
    dq = np.hypot(x - ox[minid], y - oy[minid])

    if dq <= rr:
        if dq <= 0.1:
            dq = 0.1

        return 0.5 * ETA * (1.0 / dq - 1.0 / rr) ** 2
    else:
        return 0.0

# Utility
- motion setting 
- oscillation detection
- draw heatmap

In [ ]:
def get_motion_model():
    # dx, dy
    motion = [[1, 0], [0, 1], [-1, 0], [0, -1], [-1, -1], [-1, 1], [1, -1], [1, 1]]

    return motion


def oscillations_detection(previous_ids, ix, iy):
    previous_ids.append((ix, iy))

    if len(previous_ids) > OSCILLATIONS_DETECTION_LENGTH:
        previous_ids.popleft()

    # check if contains any duplicates by copying into a set
    previous_ids_set = set()
    for index in previous_ids:
        if index in previous_ids_set:
            return True
        else:
            previous_ids_set.add(index)
    return False


def draw_heatmap(data, saveDir):
    data = np.array(data).T
    plt.pcolor(data, vmax=100.0, cmap=plt.cm.Blues)
    saveFig = os.path.join(saveDir, "headMap.jpg")
    plt.savefig(saveFig)

# planning based on potential field

In [ ]:
def potential_field_planning(sx, sy, gx, gy, ox, oy, reso, rr, saveDir):
    # calc potential field
    pmap, minx, miny = calc_potential_field(gx, gy, ox, oy, reso, rr, sx, sy)

    # search path
    d = np.hypot(sx - gx, sy - gy)
    ix = round((sx - minx) / reso)
    iy = round((sy - miny) / reso)
    gix = round((gx - minx) / reso)
    giy = round((gy - miny) / reso)

    if show_animation:
        draw_heatmap(pmap, saveDir)
        # for stopping simulation with the esc key.
        plt.gcf().canvas.mpl_connect(
            "key_release_event",
            lambda event: [exit(0) if event.key == "escape" else None],
        )
        plt.plot(ix, iy, "*k")
        plt.plot(gix, giy, "*m")

    rx, ry = [sx], [sy]
    motion = get_motion_model()
    previous_ids = deque()
    counter = 1
    while d >= reso:
        fig, ax = plt.subplots(figsize=(10, 10))
        minp = float("inf")
        minix, miniy = -1, -1
        for i, _ in enumerate(motion):
            inx = int(ix + motion[i][0])
            iny = int(iy + motion[i][1])
            if inx >= len(pmap) or iny >= len(pmap[0]) or inx < 0 or iny < 0:
                p = float("inf")  # outside area
                print("outside potential!")
            else:
                p = pmap[inx][iny]
            if minp > p:
                minp = p
                minix = inx
                miniy = iny
        ix = minix
        iy = miniy
        xp = ix * reso + minx
        yp = iy * reso + miny
        d = np.hypot(gx - xp, gy - yp)
        rx.append(xp)
        ry.append(yp)

        if oscillations_detection(previous_ids, ix, iy):
            print("Oscillation detected at ({},{})!".format(ix, iy))
            break

        if show_animation:
            ax.plot(
                rx,
                ry,
                linestyle=" ",
                color="r",
                marker="o",
                markersize=4,
                label="agent",
            )
            ax.plot(gx, gy, color="b", marker="D", markersize=8, label="goal")
            ax.plot(
                ox,
                oy,
                color="k",
                linestyle=" ",
                marker="x",
                markersize=6,
                label="obstacle",
            )
            ax.set_xlim(0, 50)
            ax.set_ylim(0, 50)
            ax.set_title(f"{counter:03d}")
            ax.legend()
            filename = os.path.join(saveDir, f"{counter:03d}.jpg")
            fig.savefig(filename)
        counter += 1

    print("Goal!!")

    return rx, ry

# Main function

In [ ]:
def main(saveDir):
    print("potential_field_planning start")

    sx = 0.0  # start x position [m]
    sy = 10.0  # start y positon [m]
    gx = 30.0  # goal x position [m]
    gy = 30.0  # goal y position [m]
    grid_size = 0.5  # potential grid size [m]
    robot_radius = 5.0  # robot radius [m]

    ox = [15.0, 5.0, 20.0, 25.0]  # obstacle x position list [m]
    oy = [25.0, 15.0, 26.0, 25.0]  # obstacle y position list [m]

    if show_animation:
        plt.grid(True)
        plt.axis("equal")

    # path generation
    _, _ = potential_field_planning(
        sx, sy, gx, gy, ox, oy, grid_size, robot_radius, saveDir
    )

    if show_animation:
        plt.show()

In [ ]:
saveDir = r"C:\Users\kawaw\python\potentialFieldMethod\fig"
if not os.path.exists(saveDir):
    os.makedirs(saveDir)
main(saveDir=saveDir)

# Make video

In [ ]:
import glob

import cv2
import os
import re


class MakeVideo:
    def __init__(self, src, videoname):
        self.src = src
        self.videoname = videoname
        self.main()

    def atoi(self, text):
        return int(text) if text.isdigit() else text

    def natural_keys(self, text):
        return [self.atoi(c) for c in re.split(r"(\d+)", text)]

    def main(self):
        img_array = []
        for filename in sorted(glob.glob(f"{self.src}/*.jpg"), key=self.natural_keys):
            img = cv2.imread(filename)
            height, width, layers = img.shape
            img_array.append(img)
        size = (width, height)
        out = cv2.VideoWriter(
            self.videoname, cv2.VideoWriter_fourcc(*"mp4v"), 3.0, size
        )

        for i in range(len(img_array)):
            out.write(img_array[i])
        out.release()

In [ ]:
src = r"C:\Users\kawaw\python\potentialFieldMethod\fig"
videoname = r"C:\Users\kawaw\python\potentialFieldMethod\fig\pathPlanning.mp4"
mkVideo = MakeVideo(src=src, videoname=videoname)